In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time 
from tqdm import tqdm


In [3]:
df = pd.read_csv("data/merged.csv")
df.head(3)

,CGG_CODE,ASGN_YY,ASGN_SNO,APPL_YMD,ASGN_YMD,UPSO_NM,SITE_ADDR_RD,SITE_ADDR,PERM_NT_NO,SNT_UPTAE_NM,MAIN_EDF,TRDP_AREA,ADMDNG_NM,GRADE_FACIL_GBN,UPSO_SITE_TELNO,UPSO_NM_MOD,UPSO_NM_LONG,SITE_ADDR_RD_SPLIT,name,url
0,3220000,2014,160,20141006,20141006,이도곰탕,"서울특별시 강남구 논현로94길 29-5, 지상1층,지상2층 (역삼동)",서울특별시 강남구 역삼동 671번지 17호,3220000-101-2001-24197,한식,곰탕,194.41,역삼1동,상수도전용,02 5010738,이도곰탕,이도곰탕 역삼1동,서울 강남구 논현로94길 29-5,이도곰탕 역삼1동,https://place.map.kakao.com/14569639
1,3220000,2021,139,20181001,20211116,솥내음 스타필드 코엑스몰점,"서울특별시 강남구 영동대로 513, 코엑스 지하1층 O-107호 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,3220000-101-2020-00643,한식,직화불백,74.56,삼성1동,NaN,NaN,솥내음 스타필드 코엑스몰,솥내음 스타필드 코엑스몰 삼성1동,서울 강남구 영동대로 513,솥내음 스타필드 코엑스몰 삼성1동,https://place.map.kakao.com/710342577
2,3220000,2010,24,20100210,20100210,스시히로바,"서울특별시 강남구 삼성로 620, 블래스톤리조트 지상1층 (삼성동)",서울특별시 강남구 삼성동 70번지 블래스톤리조트,3220000-101-2002-00383,일식,초밥,216.56,삼성1동,상수도전용,02 5155511,스시히로바,스시히로바 삼성1동,서울 강남구 삼성로 620,스시히로바 삼성1동,https://place.map.kakao.com/7974994


In [67]:
# 상세보기에 조회된 모범 음식점 목록을 조회

# store_name = "솥내음 스타필드 코엑스몰"
# page_url = "https://place.map.kakao.com/710342577"


store_names = df["UPSO_NM"]
page_urls = df["url"]
review_results = []

driver = wb.Chrome() # 브라우저 실행

for index, (store_name, page_url) in enumerate(zip(store_names, page_urls)):

    try:

        driver.get(page_url) # 음식점 상세 화면
        time.sleep(1) # 1초 쉬기


        # total_num = int((bs(driver.page_source,'lxml')).select("span.color_g")[0].text.split(")")[0][1:])
        
        soup = bs(driver.page_source,'lxml')
        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수


        more_nums = (int(tot_num.text) - 3) // 5 + 1
        print(f"--------------- {index}. {store_name} 후기 더보기 ---------------")
        for more_num in tqdm(range(more_nums)):
            try:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a')
                btnMore.click()
                time.sleep(1)
            except:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a')
                btnMore.click()
                time.sleep(1)


        soup = bs(driver.page_source,'lxml')

        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        rates = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
        users = contents_div.find_all(name="span", attrs={"class": "txt_username"})
        reviews_count = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[0::2]
        avg_rates = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[1::2]

        print(f"--------------- {index}. {store_name} 리뷰 크롤링 ---------------")

        for idx in tqdm(range(len(reviews))):

            # rate, review, user, review_count, avg_rate 지정
            user = users[idx].text
            rate = ((int(rates[idx].get("style")[6:-2]) / 100) * 5)    
            avg_rate = float(avg_rates[idx].text)
            review = reviews[idx].find(name="span").text
            review_count = reviews_count[idx].text
                
            # dict 파일 생성
            review_results.append({"stoe_name" : store_name,
                                "user" : user,                         
                                "rate" : rate,                                                    
                                "avg_rate" : avg_rate,
                                "review" : review,
                                "review_count" : review_count})
    
    except:
        print(f"카카오맵에 {store_name} 정보가 없음")

--------------- 0. 이도곰탕 후기 더보기 ---------------


100%|██████████| 34/34 [00:37<00:00,  1.11s/it]


--------------- 0. 이도곰탕 리뷰 크롤링 ---------------


100%|██████████| 169/169 [00:00<00:00, 42245.51it/s]


--------------- 1. 솥내음 스타필드 코엑스몰점 후기 더보기 ---------------


...

100%|██████████| 22/22 [00:00<00:00, 21970.16it/s]


카카오맵에 초심 정보가 없음
--------------- 317. 아야진생태찌개 역삼점 후기 더보기 ---------------


100%|██████████| 3/3 [00:03<00:00,  1.04s/it]


--------------- 317. 아야진생태찌개 역삼점 리뷰 크롤링 ---------------


100%|██████████| 17/17 [00:00<00:00, 16989.08it/s]


In [68]:
reviews_df = pd.DataFrame(review_results)

In [69]:
reviews_df

,stoe_name,user,rate,avg_rate,review,review_count
0,이도곰탕,굉장해엄청나,5.0,3.6,,434
1,이도곰탕,컹,2.0,3.0,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...,13
2,이도곰탕,🤠,5.0,4.0,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...,7
3,이도곰탕,미각세포,5.0,4.0,수육이랑 곰탕먹었어요곰탕은 주문하자마자 빨리 나왔어요고기도 많이 들어가있고간이 되어...,99
4,이도곰탕,.,5.0,3.7,세종대왕의 이름 (이도 李裪)에 걸맞는 진한 맛의 곰탕입니다마침 제옆에 외국인도 와...,171
...,...,...,...,...,...,...
8868,아야진생태찌개 역삼점,맛집수집러,2.0,3.8,처음 방문때는 생태찌개 정말 맛있게 먹었는데 이번에 재방문하니까 생태 없고 동태만 ...,18
8869,아야진생태찌개 역삼점,화남,3.0,3.3,그냥 쏘쏘.,16
8870,아야진생태찌개 역삼점,김지수,2.0,3.2,,172
8871,아야진생태찌개 역삼점,클놈,3.0,3.1,,"1,229"


In [71]:
# url 채운 파일 csv로 저장
reviews_df.to_csv("data/reviews.csv", index=False, encoding="utf-8-sig")